# Pipeline 1: Denormalize Veekun Pokédex data

The [veekun data](https://github.com/veekun/pokedex) comes in a hyper-normalized format, with subtables for every piece of data
(down to stat names and language names), so we need to convert it into a format that can be more easily digested by the ML model.

The veekun dataset includes Pokémon up to the [_Pokémon Sword and Shield Expansion Pass_](https://bulbapedia.bulbagarden.net/wiki/Pok%C3%A9mon_Sword_and_Shield_Expansion_Pass)
(from 001: Bulbasaur to 898: Calyrex). It is missing Pokémon from _Legends Arceus_ and Generation IX.

## Imports and SQLite setup

In [1]:
import sqlite3

import pandas as pd

In [2]:
POKEDEX_LOCATION = "pokedex/pokedex/data/pokedex.sqlite"

con = sqlite3.connect(POKEDEX_LOCATION)

## Extract data

### Data exploration

In [3]:
df = pd.read_sql_query("select * from pokemon_species", con)

In [4]:
df.head()

,id,identifier,generation_id,evolves_from_species_id,evolution_chain_id,color_id,shape_id,habitat_id,gender_rate,capture_rate,base_happiness,is_baby,hatch_counter,has_gender_differences,growth_rate_id,forms_switchable,is_legendary,is_mythical,order,conquest_order
0,1,bulbasaur,1,NaN,1,5,8.0,3.0,1,45,50,0,20,0,4,0,0,0,1,NaN
1,2,ivysaur,1,1.0,1,5,8.0,3.0,1,45,50,0,20,0,4,0,0,0,2,NaN
2,3,venusaur,1,2.0,1,5,8.0,3.0,1,45,50,0,20,1,4,1,0,0,3,NaN
3,4,charmander,1,NaN,2,8,6.0,4.0,1,45,50,0,20,0,4,0,0,0,4,109.0
4,5,charmeleon,1,4.0,2,8,6.0,4.0,1,45,50,0,20,0,4,0,0,0,5,110.0


In [5]:
df.shape

(898, 20)

In [6]:
pokemon_english_names = pd.read_sql_query("""
select * from pokemon_species_names
where local_language_id = 9
""", con)

In [7]:
pokemon_english_names.head()

,pokemon_species_id,local_language_id,name,genus
0,1,9,Bulbasaur,Seed Pokémon
1,2,9,Ivysaur,Seed Pokémon
2,3,9,Venusaur,Seed Pokémon
3,4,9,Charmander,Lizard Pokémon
4,5,9,Charmeleon,Flame Pokémon


### Base stats

In [8]:
pokemon_stats_long = pd.read_sql_query("""
select
    a.*,
    sp.identifier as species_name,
    st.identifier as stat_name

from pokemon_stats a
join pokemon_species sp on a.pokemon_id = sp.id
join stats st on a.stat_id = st.id
""", con)

In [9]:
pokemon_stats_long.head()

,pokemon_id,stat_id,base_stat,effort,species_name,stat_name
0,1,1,45,0,bulbasaur,hp
1,1,2,49,0,bulbasaur,attack
2,1,3,49,0,bulbasaur,defense
3,1,4,65,1,bulbasaur,special-attack
4,1,5,65,0,bulbasaur,special-defense


In [23]:
pokemon_stats_wide = (
    pd.pivot(pokemon_stats_long, index='pokemon_id', columns='stat_name', values='base_stat')
    .reset_index()
    .rename_axis(None, axis=1)
    .reindex(columns=['pokemon_id', 'hp', 'attack', 'defense', 'special-attack', 'special-defense', 'speed'])
)

In [24]:
pokemon_stats_wide.head()

,pokemon_id,hp,attack,defense,special-attack,special-defense,speed
0,1,45,49,49,65,65,45
1,2,60,62,63,80,80,60
2,3,80,82,83,100,100,80
3,4,39,52,43,60,50,65
4,5,58,64,58,80,65,80


### Height and weight

Height and weight are stored in the `pokemon` table in multiples of 0.1 kg and 0.1 m.
We convert them to meters and kilograms.

In [14]:
pokemon_height_weight = pd.read_sql_query("""
select
    species_id,
    height / 10.0 as height,
    weight / 10.0 as weight

from pokemon
where is_default = 1
""", con)

In [15]:
pokemon_height_weight.head()

,species_id,height,weight
0,1,0.7,6.9
1,2,1.0,13.0
2,3,2.0,100.0
3,4,0.6,8.5
4,5,1.1,19.0


## Join dataframes

In [31]:
joined_data = (
    df.merge(pokemon_english_names, left_on='id', right_on='pokemon_species_id')
    .merge(pokemon_stats_wide, left_on='id', right_on='pokemon_id')
    .merge(pokemon_height_weight, left_on='id', right_on='species_id')
    .drop(columns=['pokemon_species_id', 'pokemon_id', 'species_id'])
)

In [32]:
joined_data.head()

,id,identifier,generation_id,evolves_from_species_id,evolution_chain_id,color_id,shape_id,habitat_id,gender_rate,capture_rate,...,name,genus,hp,attack,defense,special-attack,special-defense,speed,height,weight
0,1,bulbasaur,1,NaN,1,5,8.0,3.0,1,45,...,Bulbasaur,Seed Pokémon,45,49,49,65,65,45,0.7,6.9
1,2,ivysaur,1,1.0,1,5,8.0,3.0,1,45,...,Ivysaur,Seed Pokémon,60,62,63,80,80,60,1.0,13.0
2,3,venusaur,1,2.0,1,5,8.0,3.0,1,45,...,Venusaur,Seed Pokémon,80,82,83,100,100,80,2.0,100.0
3,4,charmander,1,NaN,2,8,6.0,4.0,1,45,...,Charmander,Lizard Pokémon,39,52,43,60,50,65,0.6,8.5
4,5,charmeleon,1,4.0,2,8,6.0,4.0,1,45,...,Charmeleon,Flame Pokémon,58,64,58,80,65,80,1.1,19.0


## Write to Parquet

In [34]:
joined_data.to_parquet('pokemon_data.parquet')